# PARAMETROS

In [34]:
import ctypes
import tempfile
import subprocess
import os

c_code = r"""
#include <immintrin.h>

int rdseed32(unsigned int *out) {
    int ok;
    for (int i = 0; i < 10; ++i) {
        ok = _rdseed32_step(out);
        if (ok) return 1;
    }
    return 0;
}

int rdseed64(unsigned long long *out) {
    int ok;
    for (int i = 0; i < 10; ++i) {
        ok = _rdseed64_step(out);
        if (ok) return 1;
    }
    return 0;
}
"""

def compile_rdseed_lib():
    with tempfile.NamedTemporaryFile(suffix=".c", delete=False) as f:
        f.write(c_code.encode())
        c_path = f.name

    so_path = c_path.replace(".c", ".so")
    compile_cmd = [
        "gcc", "-shared", "-fPIC", "-O2", "-mrdseed", "-o", so_path, c_path
    ]

    subprocess.check_call(compile_cmd)
    os.remove(c_path)
    return ctypes.CDLL(so_path)

# Cargar lib y configurar argumentos
_rdseed = compile_rdseed_lib()
_rdseed.rdseed32.argtypes = [ctypes.POINTER(ctypes.c_uint)]
_rdseed.rdseed32.restype = ctypes.c_int

_rdseed.rdseed64.argtypes = [ctypes.POINTER(ctypes.c_ulonglong)]
_rdseed.rdseed64.restype = ctypes.c_int

# Funciones públicas
def seed32():
    out = ctypes.c_uint()
    if _rdseed.rdseed32(ctypes.byref(out)) == 0:
        raise RuntimeError("RDSEED32 falló después de varios intentos.")
    return out.value

def seed64():
    out = ctypes.c_ulonglong()
    if _rdseed.rdseed64(ctypes.byref(out)) == 0:
        raise RuntimeError("RDSEED64 falló después de varios intentos.")
    return out.value


In [190]:
# Generador Congruencial Lineal
a = 16807
M = 2**31 - 1
CGL_STATE = seed32()

# Xorshift (32,64,128)
XOR32_STATE = seed32()
XOR64_STATE = seed64()
XOR128_STATE = [seed64(), seed64()]

# Xoshiro128
XOSHIRO_STATE = [seed32(), seed32(), seed32(), seed32()]

In [36]:
from sympy import factorint

factores_primos = factorint(2**31 - 2)

print(factores_primos)


{2: 1, 3: 2, 7: 1, 11: 1, 31: 1, 151: 1, 331: 1}


In [37]:
flag = True

for p in factores_primos:
    # print(f"{pow(a, (M-1) // p, M)}")
    if pow(a, (M-1) // p, M) == 1:
        print(f"{a} NO es raíz primitiva de {M}")
        flag = False
        break

if flag:
    print(f"{a} es raíz primitiva de {M}")

16807 es raíz primitiva de 2147483647


# IMPLEMENTACIONES

## Generador Congruencial Lineal (LCG)

In [40]:
def LCG():
    """
    Generador Múltiplicativo de números pseudoaleatorios con parámetros:
    • a = 16807
    • M = 2^31 - 1 = 2147483647
    """
    global CGL_STATE
    CGL_STATE = (a*CGL_STATE) % M 
    return CGL_STATE

## Xorshifts (32,64,128)

In [ ]:
def Xorshift32():
    """
    Generador de números pseudoaleatorios de 32 bits con estado de 32 bits con parámetros:
    • a = 3
    • b = 23
    • c = 25
    """
    global XOR32_STATE
    XOR32_STATE ^= XOR32_STATE << 3 
    XOR32_STATE ^= XOR32_STATE << 25
    XOR32_STATE ^= XOR32_STATE >> 23
    XOR32_STATE &= 0xFFFFFFFF # Máscara para que sea un numero de 32bits
    return XOR32_STATE

def Xorshift64():
    """
    Generador de números pseudoaleatorios de 64 bits con estado de 64 bits con parámetros:
    • a = 1
    • b = 13
    • c = 45
    """
    global XOR64_STATE
    XOR64_STATE ^= XOR64_STATE >> 45 
    XOR64_STATE ^= XOR64_STATE << 13
    XOR64_STATE ^= XOR64_STATE >> 1
    XOR64_STATE &= 0xFFFFFFFFFFFFFFFF # Máscara para que sea un numero de 64bits
    return XOR64_STATE

def Xorshift128():
    """
    Generador de números pseudoaleatorios de 64 bits con 2 estados de 64 bits con parámetros:
    • a = 23
    • b = 18
    • c = 5
    """
    global XOR128_STATE
    s1 = XOR128_STATE[0]
    s0 = XOR128_STATE[1]
    result = (s0 + s1) & 0xFFFFFFFFFFFFFFFF # Máscara para que sea un numero de 64bits
    XOR128_STATE[0] = s1
    s1 ^= s1 << 23
    XOR128_STATE[1] = (s1 ^ s0 ^ (s1 >> 18) ^ (s0 >> 5)) & 0xFFFFFFFFFFFFFFFF # Máscara para que sea un numero de 64bits
    return result


## Xoshiro128

In [196]:
def rotl32(x, k):
    """
    Función auxiliar que calcula la rotación de k bits del número x y devuelve como resultado
    un número de 32 bits.
    """
    return ((x << k) | (x >> (32 - k))) & 0xFFFFFFFF # Máscara para que sea un numero de 32bits

def Xoshiro128():
    """
    Generador de números pseudoaleatorios de 32 bits con 4 estados de 32 bits con parámetros:
    • a = 9
    • b = 11
    • r = 7
    """
    global XOSHIRO_STATE
    result = (rotl32(XOSHIRO_STATE[0] + XOSHIRO_STATE[3], 7) + XOSHIRO_STATE[0])
    t = (XOSHIRO_STATE[1] << 9) & 0xFFFFFFFF # Máscara para que sea un numero de 32bits
    XOSHIRO_STATE[2] ^= XOSHIRO_STATE[0]
    XOSHIRO_STATE[3] ^= XOSHIRO_STATE[1]
    XOSHIRO_STATE[1] ^= XOSHIRO_STATE[2]
    XOSHIRO_STATE[0] ^= XOSHIRO_STATE[3] 
    XOSHIRO_STATE[2] ^= t
    XOSHIRO_STATE[3] =  rotl32(XOSHIRO_STATE[3], 11)
    return result

for i in range(10):
    print(Xoshiro128())
    print(f"Estado 0: {XOSHIRO_STATE[0]}")
    print(f"Estado 1: {XOSHIRO_STATE[1]}")
    print(f"Estado 2: {XOSHIRO_STATE[2]}")
    print(f"Estado 3: {XOSHIRO_STATE[3]}")

5548866735
Estado 0: 2727247554
Estado 1: 1294009636
Estado 2: 1921854676
Estado 3: 3104291416
6134893679
Estado 0: 1453900222
Estado 1: 2636273458
Estado 2: 2450070038
Estado 3: 905701281
2835181956
Estado 0: 4268847405
Estado 1: 1501793434
Estado 2: 2148515752
Estado 3: 3448020294
8480402194
Estado 0: 1786142961
Estado 1: 670908959
Estado 2: 2036857477
Estado 3: 948888736
3977849282
Estado 0: 1963312718
Estado 1: 887937131
Estado 2: 3918898292
Estado 3: 2637560059
2464508631
Estado 0: 3705464542
Estado 1: 2826746449
Estado 2: 1159771194
Estado 3: 3372516686
7743929776
Estado 0: 3181697473
Estado 1: 830531765
Estado 2: 1619587300
Estado 3: 3254319883
6653165696
Estado 0: 1306315391
Estado 1: 3970744720
Estado 2: 3707585317
Estado 3: 3231577987
2336082950
Estado 0: 1642986604
Estado 1: 2106411210
Estado 2: 3366103386
Estado 3: 2310052193
5120779105
Estado 0: 2513448391
Estado 1: 3569491452
Estado 2: 3017720118
Estado 3: 4002242465
